In [154]:
import os
import re
import pandas as pd
import numpy as np

# Configuration
LOG_BASE_PATH = "/home/jihwan/transmiter_aaai26/transmiter/logs/KET_ViT-B-16"
DATASETS = ["EuroSAT", "DTD", "Food101", "Pets", "Aircraft", "Flowers", "UCF", "Caltech", "Cars", "SUN397"]
SEEDS = [1, 2, 3]

def parse_log_file(log_path):
    """Parse the log file and extract Base and Novel accuracy from the last line."""
    try:
        with open(log_path, 'r') as f:
            lines = f.readlines()
        
        # Find the last line with "Final:" pattern
        for line in reversed(lines):
            line = line.strip()
            if line.startswith("Final:"):
                # Parse: Final: Aircraft: Base=41.72%, Novel=37.19%
                match = re.search(r'Base=(\d+\.?\d*)%.*Novel=(\d+\.?\d*)%', line)
                if match:
                    base = float(match.group(1))
                    novel = float(match.group(2))
                    return base, novel
        return None, None
    except FileNotFoundError:
        return None, None

def harmonic_mean(base, novel):
    """Calculate Harmonic Mean of Base and Novel."""
    if base + novel == 0:
        return 0
    return 2 * base * novel / (base + novel)

# Collect results
results = {}

for dataset in DATASETS:
    base_scores = []
    novel_scores = []
    
    for seed in SEEDS:
        log_path = os.path.join(LOG_BASE_PATH, dataset, str(seed), "promptsrc_logits", "log.txt")
        base, novel = parse_log_file(log_path)
        
        if base is not None and novel is not None:
            base_scores.append(base)
            novel_scores.append(novel)
            print(f"{dataset} Seed {seed}: Base={base:.2f}%, Novel={novel:.2f}%")
        else:
            print(f"{dataset} Seed {seed}: Log file not found or parse error")
    
    if base_scores and novel_scores:
        avg_base = np.mean(base_scores)
        avg_novel = np.mean(novel_scores)
        avg_hm = harmonic_mean(avg_base, avg_novel)
        results[dataset] = {
            'Base': avg_base,
            'Novel': avg_novel,
            'HM': avg_hm
        }

print("\n" + "="*50)

EuroSAT Seed 1: Base=91.81%, Novel=74.46%
EuroSAT Seed 2: Base=91.43%, Novel=72.15%
EuroSAT Seed 3: Base=94.05%, Novel=69.36%
DTD Seed 1: Base=81.25%, Novel=57.13%
DTD Seed 2: Base=81.02%, Novel=64.49%
DTD Seed 3: Base=82.99%, Novel=57.61%
Food101 Seed 1: Base=90.92%, Novel=91.95%
Food101 Seed 2: Base=91.01%, Novel=92.06%
Food101 Seed 3: Base=90.78%, Novel=90.97%
Pets Seed 1: Base=95.22%, Novel=92.62%
Pets Seed 2: Base=95.85%, Novel=97.87%
Pets Seed 3: Base=95.37%, Novel=97.99%
Aircraft Seed 1: Base=39.38%, Novel=37.61%
Aircraft Seed 2: Base=40.94%, Novel=37.91%
Aircraft Seed 3: Base=39.80%, Novel=38.69%
Flowers Seed 1: Base=98.10%, Novel=78.23%
Flowers Seed 2: Base=98.10%, Novel=78.51%
Flowers Seed 3: Base=98.20%, Novel=78.94%
UCF Seed 1: Base=85.63%, Novel=80.10%
UCF Seed 2: Base=85.73%, Novel=79.66%
UCF Seed 3: Base=86.04%, Novel=79.02%
Caltech Seed 1: Base=97.93%, Novel=94.10%
Caltech Seed 2: Base=97.93%, Novel=94.00%
Caltech Seed 3: Base=97.87%, Novel=95.09%
Cars Seed 1: Base=78.3

In [155]:
# Create DataFrame and display table
df = pd.DataFrame(results).T
df = df.round(2)

# Reorder columns
df = df[['Base', 'Novel', 'HM']]

# Add average row
avg_row = pd.DataFrame({
    'Base': [df['Base'].mean()],
    'Novel': [df['Novel'].mean()],
    'HM': [df['HM'].mean()]
}, index=['Average'])

df = pd.concat([df, avg_row])
df = df.round(2)

print("\nKET_ViT-B-16 Results (Seed Average)")
print("="*50)
display(df)


KET_ViT-B-16 Results (Seed Average)


,Base,Novel,HM
EuroSAT,92.43,71.99,80.94
DTD,81.75,59.74,69.04
Food101,90.90,91.66,91.28
Pets,95.48,96.16,95.82
Aircraft,40.04,38.07,39.03
Flowers,98.13,78.56,87.26
UCF,85.80,79.59,82.58
Caltech,97.91,94.40,96.12
Cars,78.17,76.22,77.18
SUN397,81.87,79.49,80.66
